In [1]:
import sys
sys.path.insert(0, '../')
import fconcrete as fc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Tamanho da figura a ser plotada
plt.rcParams["figure.figsize"] = (8,8)

In [33]:
>>> def criar_viga_sem_remover_barras(base, altura, comprimento):
...        area_laje = comprimento*comprimento
...        carga_na_laje = fc.to_unit(5, "kN/m**2", "kN/cm**2")
...        carga_na_viga = -(carga_na_laje*area_laje/comprimento)/4
...        carga_distribuida_na_viga = fc.Load.UniformDistributedLoad(carga_na_viga, x_begin=0, x_end=comprimento)
...        peso_proprio = fc.Load.UniformDistributedLoad(-base*altura*25/1000000, x_begin=0, x_end=comprimento)
...        n1 = fc.Node.SimpleSupport(x=0)
...        n2 = fc.Node.SimpleSupport(x=comprimento)
...        beam = fc.ConcreteBeam(
...            loads = [carga_distribuida_na_viga],
...            nodes = [n1, n2],
...            section = fc.Rectangle(base, altura),
...            bar_steel_max_removal = 0,
...        )
...        return beam


In [107]:
concreto

839.09

In [18]:
viga_removendo_barras = criar_viga(25, 95, 1000, 999999)

,0,1,2,3,4,5
0,Material,Price,Quantity,Unit,Commentary,Is Subtotal
1,Concrete,839.09,2.38,m3,,True
2,Longitudinal bar,338.91,6225.76,m,,True
3,Transversal bar,176.43,8575.0,m,,True


In [112]:
dados_menores_custos = pd.read_excel("Dados de custo das melhores dimensoes.xlsx")

In [116]:
dados_menores_custos

,base,altura,comprimento,custo,concreto,barras longitudinais,barras transversais
150,15,15,150,26.552389,11.92,6.60,8.02
200,15,15,200,35.238586,15.90,8.64,10.70
250,15,15,250,43.924782,19.87,10.68,13.37
300,15,19,300,61.904167,30.21,15.18,16.52
350,15,25,350,88.298648,46.37,19.19,22.74
400,15,29,400,119.126072,61.47,28.95,28.70
450,15,35,450,154.924409,83.47,36.89,34.57
500,15,41,500,201.408929,108.64,49.44,43.33
550,15,47,550,247.964731,136.99,57.89,53.08
600,15,53,600,300.256021,168.52,70.81,60.92


In [136]:
def calculos_novos_custos(row):
    viga_sem_remover_barras = criar_viga_sem_remover_barras(row["base"], row["altura"], row["comprimento"])
    tabela_subtotal = viga_sem_remover_barras.subtotal_table
    columns = list(tabela_subtotal[0, :])
    tabela_com_materiais_e_preco = pd.DataFrame(viga_sem_remover_barras.subtotal_table, columns=columns).iloc[1:, 0:2]
    tabela_com_materiais_e_preco["Price"] = tabela_com_materiais_e_preco["Price"].apply(float)
    tabela_com_materiais_e_preco = tabela_com_materiais_e_preco.pivot_table("Price", None, "Material")
    concreto, barras_longitudinais, barras_transversais = tabela_com_materiais_e_preco.values[0]
    custo_total = concreto + barras_longitudinais + barras_transversais
    return custo_total, concreto, barras_longitudinais, barras_transversais

In [137]:
coluna_sem_retirada = dados_menores_custos.apply(calculos_novos_custos, axis=1)

In [139]:
dados_menores_custos["custo total sem retirada"] = coluna_sem_retirada.apply(lambda x: x[0])

In [140]:
dados_menores_custos["concreto sem retirada"] = coluna_sem_retirada.apply(lambda x: x[1])
dados_menores_custos["barras longitudinais sem retirada"] = coluna_sem_retirada.apply(lambda x: x[2])
dados_menores_custos["barras trasnversais sem retirada"] = coluna_sem_retirada.apply(lambda x: x[3])

In [144]:
dados_menores_custos

,base,altura,comprimento,custo,concreto,barras longitudinais,barras transversais,custo total sem retirada,concreto sem retirada,barras longitudinais sem retirada,barras trasnversais sem retirada
150,15,15,150,26.552389,11.92,6.60,8.02,26.59,11.92,6.65,8.02
200,15,15,200,35.238586,15.90,8.64,10.70,35.31,15.90,8.71,10.70
250,15,15,250,43.924782,19.87,10.68,13.37,44.00,19.87,10.76,13.37
300,15,19,300,61.904167,30.21,15.18,16.52,66.15,30.21,19.42,16.52
350,15,25,350,88.298648,46.37,19.19,22.74,91.90,46.37,22.79,22.74
400,15,29,400,119.126072,61.47,28.95,28.70,124.93,61.47,34.76,28.70
450,15,35,450,154.924409,83.47,36.89,34.57,167.12,83.47,49.08,34.57
500,15,41,500,201.408929,108.64,49.44,43.33,217.62,108.64,65.65,43.33
550,15,47,550,247.964731,136.99,57.89,53.08,262.47,136.99,72.40,53.08
600,15,53,600,300.256021,168.52,70.81,60.92,321.79,168.52,92.35,60.92


In [143]:
dados_menores_custos_simplificado = dados_menores_custos[["base", "altura", "comprimento", "custo", "barras longitudinais", "custo total sem retirada", "barras longitudinais sem retirada"]]
dados_menores_custos_simplificado


,base,altura,comprimento,custo,barras longitudinais,custo total sem retirada,barras longitudinais sem retirada
150,15,15,150,26.552389,6.60,26.59,6.65
200,15,15,200,35.238586,8.64,35.31,8.71
250,15,15,250,43.924782,10.68,44.00,10.76
300,15,19,300,61.904167,15.18,66.15,19.42
350,15,25,350,88.298648,19.19,91.90,22.79
400,15,29,400,119.126072,28.95,124.93,34.76
450,15,35,450,154.924409,36.89,167.12,49.08
500,15,41,500,201.408929,49.44,217.62,65.65
550,15,47,550,247.964731,57.89,262.47,72.40
600,15,53,600,300.256021,70.81,321.79,92.35


In [175]:
dados_menores_custos_relacoes = dados_menores_custos_simplificado.copy()
reducao_em_percentagem_total = 100*(dados_menores_custos_relacoes["custo"]-dados_menores_custos_relacoes["custo total sem retirada"])/dados_menores_custos_relacoes["custo"]
dados_menores_custos_relacoes["Redução do valor total"] = reducao_em_percentagem_total.round(3).apply(str) + "%"

reducao_em_percentagem_de_barras_longitudinais = 100*(dados_menores_custos_relacoes["barras longitudinais"]-dados_menores_custos_relacoes["barras longitudinais sem retirada"])/dados_menores_custos_relacoes["barras longitudinais"]
dados_menores_custos_relacoes["Redução do valor de barras longitudinais"] = reducao_em_percentagem_de_barras_longitudinais.round(3).apply(str) + "%"


In [176]:
dados_menores_custos_relacoes[["base", "altura", "comprimento", "Redução do valor total", "Redução do valor de barras longitudinais"]]


,base,altura,comprimento,Redução do valor total,Redução do valor de barras longitudinais
150,15,15,150,-0.142%,-0.758%
200,15,15,200,-0.203%,-0.81%
250,15,15,250,-0.171%,-0.749%
300,15,19,300,-6.859%,-27.931%
350,15,25,350,-4.079%,-18.76%
400,15,29,400,-4.872%,-20.069%
450,15,35,450,-7.872%,-33.044%
500,15,41,500,-8.049%,-32.787%
550,15,47,550,-5.85%,-25.065%
600,15,53,600,-7.172%,-30.419%
